In [1]:
import sys
import pandas as pd
import numpy as np
import pickle
import json

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.ensemble import RandomForestClassifier

In [2]:
model = pickle.load(open('../data/model.sav', 'rb'))
# path = sys.argv[1]
# name = sys.argv[2]

# path = '../task2/A549_directRNA_replicate5_run1.json'
# name = 'pred_A549_replicate5_run1'

# path = '../task2/A549_directRNA_replicate6_run1.json'
# name = 'pred_A549_replicate6_run1'

# path = '../task2/Hct116_directRNA_replicate3_run1.json'
# name = 'pred_Hct116_replicate3_run1'

# path = '../task2/Hct116_directRNA_replicate3_run4.json'
# name = 'pred_Hct116_replicate3_run4'

# path = '../task2/Hct116_directRNA_replicate4_run3.json'
# name = 'pred_Hct116_replicate4_run3'

# path = '../task2/HepG2_directRNA_replicate5_run2.json'
# name = 'pred_HepG2_replicate5_run2'

# path = '../task2/HepG2_directRNA_replicate6_run1.json'
# name = 'pred_HepG2_replicate6_run1'

# path = '../task2/K562_directRNA_replicate4_run1.json'
# name = 'pred_K562_replicate4_run1'

# path = '../task2/K562_directRNA_replicate5_run1.json'
# name = 'pred_K562_replicate5_run1'

# path = '../task2/K562_directRNA_replicate6_run1.json'
# name = 'pred_K562_replicate6_run1'

# path = '../task2/MCF7_directRNA_replicate3_run1.json'
# name = 'pred_MCF7_replicate3_run1'

path = '../task2/MCF7_directRNA_replicate4_run1.json'
name = 'pred_MCF7_replicate4_run1'

In [3]:
## read_json.ipynb ##

# find the number of occurrence of a letter in a word
# input: str word, str letter
# output: int
def find(word, letter):
    res = 0
    for i in word:
        if i==letter:
            res += 1
    return res

# normalise the numerical columns
# input: df
# output: normalised df
def normalise(df):
    numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns
    string_columns = df.select_dtypes(include=['object', 'category']).columns

    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(df[numerical_columns])

    df_normalised = pd.DataFrame(x_scaled)
    df_normalised.columns = numerical_columns

    final_df = pd.concat([df[string_columns].reset_index(), df_normalised], axis=1)
    final_df = final_df.drop(columns = ['index'])
    return final_df

def join_pred(df, y_pred):
    new_df = pd.concat([df[['transcript_id', 'transcript_position']], pd.DataFrame(y_pred)], axis=1)
    new_df = new_df.rename(columns = {0: 'score'})
    return new_df

# return the key and value of the dictionary separately
# input: dictionary
# output: key, value
def split(x):
    key = list(x.keys())[0]
    value = x.get(key)
    return key, value


In [4]:
with open(path, "r") as read_file:
    data = [json.loads(l) for l in read_file]

transcript_list, position_list, sevenmers_list, reads_list = [], [], [], []
for line in data:
    t, dic = split(line)
    p, dic = split(dic)
    s, reads = split(dic)
    
    for read in reads:
        transcript_list.append(t)
        position_list.append(p)
        sevenmers_list.append(s)
        reads_list.append(read)

df = pd.DataFrame.from_dict({'transcript_id': transcript_list, 
                             'transcript_position': position_list, 
                             'sevenmers': sevenmers_list, 
                             'reads': reads_list})

df1 = pd.DataFrame(df['reads'].to_list())
df3 = pd.concat([df.reset_index(), df1], axis=1)
df3 = df3.rename(columns = {0:'dwelling_time_1', 1:'sd_current_1', 2:'mean_current_1',
                            3:'dwelling_time_2', 4:'sd_current_2', 5:'mean_current_2',
                            6:'dwelling_time_3', 7:'sd_current_3', 8:'mean_current_3'})

final_df = df3.drop(columns=['reads', 'index'])
final_df

,transcript_id,transcript_position,sevenmers,dwelling_time_1,sd_current_1,mean_current_1,dwelling_time_2,sd_current_2,mean_current_2,dwelling_time_3,sd_current_3,mean_current_3
0,R2_55_2,37,GAAACTG,0.002320,3.156000,105.5,0.004650,2.683000,109.4,0.009261,2.397481,95.3
1,R2_55_2,37,GAAACTG,0.024240,1.817000,111.4,0.009928,2.116471,111.2,0.003650,1.946000,94.2
2,R2_55_2,45,TAAACTA,0.004320,1.490000,109.5,0.006063,1.826267,106.3,0.003556,1.345143,94.9
3,R2_55_2,45,TAAACTA,0.007514,3.219233,105.6,0.004650,2.211000,106.1,0.004002,1.538194,94.5
4,R2_55_2,226,CAAACAA,0.003650,2.125000,107.8,0.002320,3.190000,101.3,0.004400,2.473348,88.7
...,...,...,...,...,...,...,...,...,...,...,...,...
15671881,R2_66_2,769,TAAACAT,0.010943,2.665984,108.6,0.003320,3.555000,103.9,0.004781,2.776370,86.2
15671882,R2_66_2,769,TAAACAT,0.004434,2.230783,107.2,0.001990,2.924000,99.7,0.017142,3.581685,89.1
15671883,R2_66_2,769,TAAACAT,0.005491,2.885782,105.8,0.002990,2.177000,101.6,0.013940,4.044000,87.8
15671884,R2_66_2,769,TAAACAT,0.006369,2.617853,105.8,0.003980,5.933000,101.0,0.003776,3.205636,84.9


In [5]:

## feature_engineering.ipynb ##
df = final_df
# get the difference of each features 
df['diff_dwelling_time_1'] = df['dwelling_time_2'] - df['dwelling_time_1']
df['diff_dwelling_time_2'] = df['dwelling_time_3'] - df['dwelling_time_2']
df['diff_sd_current_1'] = df['sd_current_2'] - df['sd_current_1']
df['diff_sd_current_2'] = df['sd_current_3'] - df['sd_current_2']
df['diff_mean_current_1'] = df['mean_current_2'] - df['mean_current_1']
df['diff_mean_current_2'] = df['mean_current_3'] - df['mean_current_2']
# aggregate and find min, max, median, std for each features

lst = ['transcript_id', 'transcript_position', 'sevenmers']

grouped_df = df.groupby(by = lst).agg(
    {'dwelling_time_1': [min, max, 'median', 'std'],
     'sd_current_1': [min, max, 'median', 'std'],
     'mean_current_1': [min, max, 'median', 'std'],
     'dwelling_time_2': [min, max, 'median', 'std'],
     'sd_current_2': [min, max, 'median', 'std'],
     'mean_current_2': [min, max, 'median', 'std'],
     'dwelling_time_3': [min, max, 'median', 'std'],
     'sd_current_3': [min, max, 'median', 'std'],
     'mean_current_3': [min, max, 'median', 'std'],
     'diff_dwelling_time_1': [min, max, 'median', 'std'],
     'diff_dwelling_time_2': [min, max, 'median', 'std'],
     'diff_sd_current_1': [min, max, 'median', 'std'],
     'diff_sd_current_2': [min, max, 'median', 'std'],
     'diff_mean_current_1': [min, max, 'median', 'std'],
     'diff_mean_current_2': [min, max, 'median', 'std'],
    }).reset_index()

# rename the columns
grouped_df2 = grouped_df
grouped_df2.columns = ["_".join(x) for x in np.ravel(grouped_df.columns)]
grouped_df2 = grouped_df.rename(columns = {'transcript_id_': 'transcript_id', 
                                           'transcript_position_': 'transcript_position',
                                           'sevenmers_': 'sevenmers'})
grouped_df2 = grouped_df2.fillna(value=0)    # fill the NA values with 0

# find the relative position of each read in each transcript
grouped_df2['transcript_position'] = grouped_df2['transcript_position'].astype(int)
grouped_df2['relative_position'] = grouped_df2.groupby('transcript_id')['transcript_position'].transform(lambda x: (x - x.min())/(x.max()-x.min()))

# note: have NAs because there's transcripts with only one position
# fill the NAs with 0
grouped_df2['relative_position'] = grouped_df2['relative_position'].fillna(0)

# split the sevenmers into seven columns
order_df = pd.DataFrame(grouped_df2['sevenmers'].str.split('').to_list())[[x for x in range(1, 8)]].rename(
    columns = {1: 'order_1', 2:'order_2', 
               3: 'order_3', 4:'order_4', 
               5: 'order_5', 6:'order_6',
               7: 'order_7'})
grouped_df3 = pd.concat([grouped_df2, order_df], axis = 1)

# order_4 is always A, order_5 is always C --> drop
grouped_df3 = grouped_df3.drop(columns=['order_4', 'order_5'])

# count the A,C,G,T in the sevenmers
grouped_df3['count_A'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'A'))
grouped_df3['count_C'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'C'))
grouped_df3['count_G'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'G'))
grouped_df3['count_T'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'T'))
grouped_df3

,transcript_id,transcript_position,sevenmers,dwelling_time_1_min,dwelling_time_1_max,dwelling_time_1_median,dwelling_time_1_std,sd_current_1_min,sd_current_1_max,sd_current_1_median,...,relative_position,order_1,order_2,order_3,order_6,order_7,count_A,count_C,count_G,count_T
0,ENST00000000233,244,AAGACCA,0.001990,0.034200,0.006310,0.005559,1.331000,13.415112,3.649500,...,0.000000,A,A,G,C,A,4,2,1,0
1,ENST00000000233,261,CAAACTG,0.001990,0.025560,0.005784,0.004552,0.848000,11.267000,3.108000,...,0.025411,C,A,A,T,G,3,2,1,1
2,ENST00000000233,316,GAAACAG,0.002320,0.025230,0.007593,0.004389,1.209000,12.258000,2.735000,...,0.107623,G,A,A,A,G,4,1,2,0
3,ENST00000000233,332,AGAACAT,0.001990,0.042152,0.008535,0.004587,1.797000,14.453000,5.918588,...,0.131540,A,G,A,A,T,4,1,1,1
4,ENST00000000233,368,AGGACAA,0.001990,0.056110,0.008630,0.007408,1.459000,10.461472,6.293711,...,0.185351,A,G,G,A,A,4,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446017,R2_7_2,798,CAGACCT,0.002990,0.007640,0.005315,0.003288,3.332000,4.779000,4.055500,...,0.456258,C,A,G,C,T,2,3,1,1
1446018,R2_7_2,828,ATGACCG,0.005111,0.017260,0.011186,0.008591,3.772565,4.249000,4.010783,...,0.474484,A,T,G,C,G,2,2,2,1
1446019,R2_7_2,885,AGAACTA,0.011905,0.015929,0.013917,0.002846,3.576238,3.852137,3.714188,...,0.509113,A,G,A,T,A,4,1,1,1
1446020,R2_7_2,904,CGAACCG,0.004980,0.006640,0.005810,0.001174,8.807000,9.721000,9.264000,...,0.520656,C,G,A,C,G,2,3,2,0


In [6]:

## resample_normalise.ipynb ##
df = grouped_df3
features_nominal = ['order_1', 'order_2', 'order_3', 'order_6', 'order_7']
df[features_nominal] = df[features_nominal].astype('category')

df_norm = normalise(df)
df_norm['transcript_position'] = df['transcript_position']

## feature_selection.ipynb ##
# one hot encode the categories
features_nominal = ['order_1', 'order_2', 'order_3', 'order_6', 'order_7']
df_norm = pd.get_dummies(df_norm, columns = features_nominal)
df = pd.get_dummies(df, columns = features_nominal)

selected = pickle.load(open('../data/features.info', 'rb'))
df_norm = df_norm[selected] # normalised version
df_selected = df[selected] # un-normalised version

# make predictions
y_pred = model.predict_proba(df_norm)[:, 1]

y_df = join_pred(df, y_pred)
df = df.drop(columns = ['transcript_id', 'transcript_position'])
output_df = pd.concat([y_df, df], axis=1)
output_df

,transcript_id,transcript_position,score,sevenmers,dwelling_time_1_min,dwelling_time_1_max,dwelling_time_1_median,dwelling_time_1_std,sd_current_1_min,sd_current_1_max,...,order_2_T,order_3_A,order_3_G,order_6_A,order_6_C,order_6_T,order_7_A,order_7_C,order_7_G,order_7_T
0,ENST00000000233,244,0.170714,AAGACCA,0.001990,0.034200,0.006310,0.005559,1.331000,13.415112,...,0,0,1,0,1,0,1,0,0,0
1,ENST00000000233,261,0.241682,CAAACTG,0.001990,0.025560,0.005784,0.004552,0.848000,11.267000,...,0,1,0,0,0,1,0,0,1,0
2,ENST00000000233,316,0.150655,GAAACAG,0.002320,0.025230,0.007593,0.004389,1.209000,12.258000,...,0,1,0,1,0,0,0,0,1,0
3,ENST00000000233,332,0.173953,AGAACAT,0.001990,0.042152,0.008535,0.004587,1.797000,14.453000,...,0,1,0,1,0,0,0,0,0,1
4,ENST00000000233,368,0.206741,AGGACAA,0.001990,0.056110,0.008630,0.007408,1.459000,10.461472,...,0,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446017,R2_7_2,798,0.185034,CAGACCT,0.002990,0.007640,0.005315,0.003288,3.332000,4.779000,...,0,0,1,0,1,0,0,0,0,1
1446018,R2_7_2,828,0.104172,ATGACCG,0.005111,0.017260,0.011186,0.008591,3.772565,4.249000,...,1,0,1,0,1,0,0,0,1,0
1446019,R2_7_2,885,0.298223,AGAACTA,0.011905,0.015929,0.013917,0.002846,3.576238,3.852137,...,0,1,0,0,0,1,1,0,0,0
1446020,R2_7_2,904,0.159756,CGAACCG,0.004980,0.006640,0.005810,0.001174,8.807000,9.721000,...,0,1,0,0,1,0,0,0,1,0


In [7]:
output_df

,transcript_id,transcript_position,score,sevenmers,dwelling_time_1_min,dwelling_time_1_max,dwelling_time_1_median,dwelling_time_1_std,sd_current_1_min,sd_current_1_max,...,order_2_T,order_3_A,order_3_G,order_6_A,order_6_C,order_6_T,order_7_A,order_7_C,order_7_G,order_7_T
0,ENST00000000233,244,0.170714,AAGACCA,0.001990,0.034200,0.006310,0.005559,1.331000,13.415112,...,0,0,1,0,1,0,1,0,0,0
1,ENST00000000233,261,0.241682,CAAACTG,0.001990,0.025560,0.005784,0.004552,0.848000,11.267000,...,0,1,0,0,0,1,0,0,1,0
2,ENST00000000233,316,0.150655,GAAACAG,0.002320,0.025230,0.007593,0.004389,1.209000,12.258000,...,0,1,0,1,0,0,0,0,1,0
3,ENST00000000233,332,0.173953,AGAACAT,0.001990,0.042152,0.008535,0.004587,1.797000,14.453000,...,0,1,0,1,0,0,0,0,0,1
4,ENST00000000233,368,0.206741,AGGACAA,0.001990,0.056110,0.008630,0.007408,1.459000,10.461472,...,0,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446017,R2_7_2,798,0.185034,CAGACCT,0.002990,0.007640,0.005315,0.003288,3.332000,4.779000,...,0,0,1,0,1,0,0,0,0,1
1446018,R2_7_2,828,0.104172,ATGACCG,0.005111,0.017260,0.011186,0.008591,3.772565,4.249000,...,1,0,1,0,1,0,0,0,1,0
1446019,R2_7_2,885,0.298223,AGAACTA,0.011905,0.015929,0.013917,0.002846,3.576238,3.852137,...,0,1,0,0,0,1,1,0,0,0
1446020,R2_7_2,904,0.159756,CGAACCG,0.004980,0.006640,0.005810,0.001174,8.807000,9.721000,...,0,1,0,0,1,0,0,0,1,0


In [8]:
output_df.to_csv(f'../task2/{name}.csv', index=False)
print(f'Execution successful! Prediction can now be found at ../task2/{name}.csv')
print()

Execution successful! Prediction can now be found at ../task2/pred_MCF7_replicate4_run1.csv

